In [1]:
import pandas as pd
import numpy as np
from tkinter import *
from tkinter import ttk
import tkinter as tk
import tkinter.filedialog
import datetime
import time

In [2]:
line1_workers = 8
line2_workers = 5
combination_n = int(input('pls input n:'))
#combination_n = int(combination_n)
ratio = float(input('pls input ratio (0.5-1):'))

pls input n:100
pls input ratio (0.5-1):0.9


In [3]:
window = tk.Tk()
window.geometry('400x400')
window.title('Line Balance')

''

In [4]:
def open_file1():
    global filename1
    filename1 = tkinter.filedialog.askopenfilename()

def open_file2():
    global filename2
    filename2 = tkinter.filedialog.askopenfilename()   
    
def open_file3():
    global filename3
    filename3 = tkinter.filedialog.askopenfilename()   


def submit_move_H():
    global df1
    global df2
    global df3
    
    if (filename1.split('.')[1].upper() == 'XLSX') | (filename1.split('.')[1].upper() == 'XLSM'):
        df1 = pd.read_excel(filename1)
    elif filename1.split('.')[1].upper() == 'CSV':
        df1 = pd.read_csv(filename1)
    df2 = pd.read_excel(filename2)
    df3 = pd.read_excel(filename3)
    prep()
    move_H()
    #window.destroy()

def submit_move_MH():
    global df1
    global df2
    global df3
    
    if (filename1.split('.')[1].upper() == 'XLSX') | (filename1.split('.')[1].upper() == 'XLSM'):
        df1 = pd.read_excel(filename1)
    elif filename1.split('.')[1].upper() == 'CSV':
        df1 = pd.read_csv(filename1)
    df2 = pd.read_excel(filename2)
    df3 = pd.read_excel(filename3)
    prep()
    move_H()
    move_MH()
    #window.destroy()

def submit_move_M():
    global df1
    global df2
    global df3
    
    if (filename1.split('.')[1].upper() == 'XLSX') | (filename1.split('.')[1].upper() == 'XLSM'):
        df1 = pd.read_excel(filename1)
    elif filename1.split('.')[1].upper() == 'CSV':
        df1 = pd.read_csv(filename1)
    df2 = pd.read_excel(filename2)
    df3 = pd.read_excel(filename3)
    prep()
    move_H()
    move_MH()
    move_M()
    #window.destroy()

def submit_move_ML():
    global df1
    global df2
    global df3
    
    if (filename1.split('.')[1].upper() == 'XLSX') | (filename1.split('.')[1].upper() == 'XLSM'):
        df1 = pd.read_excel(filename1)
    elif filename1.split('.')[1].upper() == 'CSV':
        df1 = pd.read_csv(filename1)
    df2 = pd.read_excel(filename2)
    df3 = pd.read_excel(filename3)
    prep()
    move_H()
    move_MH()
    move_M()
    move_ML()
    #window.destroy()

In [5]:
def prep():
    global df1
    global df2
    global df3
    global df
    global H_value
    global MH_value
    global M_value
    global ML_value
    global L_value
    #df0 = pd.read_excel('C:\\Users\\chenmin\\Desktop\\Summary\\All Bin Code3.xlsx')
    df0 = pd.read_excel('./All Bin Code3.xlsx')
    #df00 = pd.read_excel('C:\\Users\\chenmin\\Desktop\\Summary\\All product Dimension.xlsx')
    df00 = pd.read_excel('./All Product Dimension.xlsx')
    df00.rename(columns={'Product':'Material'},inplace=True)
    
    df1 = df1[['Material','Description','Confirmed Quantity']][(df1['Sales Document Type']==('ZOR')) | (df1['Sales Document Type']==('ZOR1')) | (df1['Sales Document Type']==('ZOR3')) | (df1['Sales Document Type']==('YPF3')) | (df1['Sales Document Type']==('YPFD'))]
    df1.to_csv('material.csv',index=False)
    df1 = pd.read_csv('material.csv')
    df1 = df1.groupby(by='Material').agg({'Confirmed Quantity':sum}).reset_index()
    df1 = df1.sort_values(by='Confirmed Quantity',ascending = False) 
    df1.rename(columns={'Confirmed Quantity':'Qty'}, inplace=True)
    df1[['Material']] = df1[['Material']].astype(float)

    df2.rename(columns={'Product':'Material','Storage Bin':'Bin'},inplace=True)
    df2[['Material']] = df2[['Material']].astype(float)

    cols = df3.iloc[5,[5,6,7,15,16,17]]
    df3 = df3.iloc[6:,[5,6,7,15,16,17]]
    df3.rename(columns = cols, inplace=True)
    df3.rename(columns={'Material code':'Material'},inplace=True)
    droplist = df3[(df3['Description']=='Varsha')|\
              (df3['Material']=='2019 RJMB ADV SPECIAL DECANTER 100ML')|\
              (df3['Material']=='NEW 2019 LARGE SIGNATURE BLACK TOTE')|\
              (df3['Material']=='NEW 2019 MEDIUM SIGNATURE BLACK TOTE')|\
              (df3['Material']=='WN_Royal Man De-tox Gel Cleanser')].index.tolist()
    df3 = df3.drop(droplist)
    df3[['Material']] = df3[['Material']].astype(float)
    #df3.to_csv('C:\\Users\\chenmin\\Desktop\\Summary\\All material from Exp.csv',index=False)
    df3.to_csv('./All material from Exp.csv', index=False)
    
    # merge and reorder columns
    df = pd.merge(df0,df2[['Bin','Material']],on='Bin',how='left').merge(df1,on='Material',how='left').merge(df3[['Material','Description']],on='Material',how='left').merge(df00[['Material','Volume']],on='Material',how='left')
    df
    cols = list(df)
    print(cols)
    cols.insert(4,cols.pop(cols.index('Description')))
    cols.insert(5,cols.pop(cols.index('Volume')))
    cols.insert(6,cols.pop(cols.index('Worker')))
    cols.insert(7,cols.pop(cols.index('Bin')))
    cols.insert(8,cols.pop(cols.index('BinRank')))
    print(cols)
    df = df.loc[:,cols]
    df.fillna(0,inplace=True)
    df.head()

    # insert columns: NewWorker, NewBin, NewBinRank, Change
    df.insert(7,'NewWorker-H',0)
    df.insert(8,'NewBin-H',0)
    df.insert(9,'NewBinRank-H',0)
    df.insert(10,'Change-H',0)

    for row in range(df.shape[0]):
        df.iloc[row,7] = df.iloc[row,4]
        df.iloc[row,8] = df.iloc[row,5]
        df.iloc[row,9] = df.iloc[row,6]
    
    col = df.columns.values.tolist()

    H_num = df[df['BinRank'] == 'high'].shape[0]
    MH_num = df[df['BinRank'] == 'medium high'].shape[0]
    M_num = df[df['BinRank'] == 'medium'].shape[0]
    ML_num = df[df['BinRank'] == 'medium low'].shape[0]
    L_num = df[df['BinRank'] == 'low'].shape[0]
    print('High:','%10s'%H_num,'bins')
    print('Medium_High:','%3s'%MH_num,'bins')
    print('Medium:','%8s'%M_num,'bins')
    print('Medium_Low:','%4s'%ML_num,'bins')
    print('Low:','%11s'%L_num,'bins')
    print()

    qty_list = df['Qty'].tolist()
    qty_list.sort(reverse=True)
    H_value = qty_list[H_num-1]
    MH_value = qty_list[H_num + MH_num -1]
    M_value = qty_list[H_num + MH_num + M_num -1]
    ML_value = qty_list[H_num + MH_num + M_num + ML_num -1]
    print('High:','%12s'%'Qty>=',H_value)
    print('Medium_High:','%5s'%'Qty>=',MH_value)
    print('Medium:','%10s'%'Qty>=',M_value)
    print('Medium_Low:','%6s'%'Qty>=',ML_value)
    print('Low:','%12s'%'Qty<',ML_value)
    df.to_csv('test.csv',index=False)
    
    

In [6]:
# high SKUs
def move_H():
    
    add_idx = df[(df['BinRank']!='high')&(df['Qty']>=H_value)].index.tolist()
    remove_idx = df[(df['BinRank']=='high')&(df['Qty']<H_value)].index.tolist()
    print('add_idx:   ',add_idx)
    print('Total add:',len(add_idx),'\n')
    print('remove_idx:',remove_idx)
    print('Total remove:',len(remove_idx),'\n')

    pop_n = abs(len(add_idx) - len(remove_idx))
    while pop_n != 0:
        if len(add_idx) > len(remove_idx):
            add_idx.pop()
            pop_n -= 1
        if len(add_idx) < len(remove_idx):
            remove_idx.pop()
            pop_n -= 1
    print('add_idx:   ',add_idx)
    print('Total add:',len(add_idx),'\n')
    print('remove_idx:',remove_idx)
    print('Total remove:',len(remove_idx),'\n') 

    # find most suitable bin for high SKUs
    add_idx_final = []

    avg_1 = df[df['Qty']>=H_value]['Qty'].sum()/line1_workers

    df_group = df[df['Qty']>=H_value].groupby(by=('Worker'))['Qty'].sum()
    print(df_group)

    target = (abs(avg_1-df_group['1A'])+abs(avg_1-df_group['1B'])+abs(avg_1-df_group['1C'])+abs(avg_1-df_group['1D'])+\
             abs(avg_1-df_group['1E'])+abs(avg_1-df_group['1F'])+abs(avg_1-df_group['1G'])+abs(avg_1-df_group['1H']))/(line1_workers)
    print('target:',target,'\n')

    n = combination_n
    while n > 0:
        np.random.shuffle(add_idx)
        #print('remove_idx',remove_idx)

        for (i,j) in zip(add_idx,remove_idx):
            df.iloc[i,7] = df.iloc[j,4]
            df.iloc[i,8] = df.iloc[j,5]
            df.iloc[i,9] = df.iloc[j,6]
            df.iloc[j,7] = df.iloc[i,4]
            df.iloc[j,8] = df.iloc[i,5]
            df.iloc[j,9] = df.iloc[i,6]
    
        df_group = df[df['Qty']>=H_value].groupby(by=('NewWorker-H'))['Qty'].sum()
        #print(df_group)
        target_temp = (abs(avg_1-df_group['1A'])+abs(avg_1-df_group['1B'])+abs(avg_1-df_group['1C'])+abs(avg_1-df_group['1D'])+\
                      abs(avg_1-df_group['1E'])+abs(avg_1-df_group['1F'])+abs(avg_1-df_group['1G'])+abs(avg_1-df_group['1H']))/(line1_workers)
        #print('target_temp:',target_temp)
        if target_temp < target:
            target = target_temp
            #print('target:',target)
            add_idx_final = add_idx.copy()
            #print('add_idx_final:',add_idx_final)
            print(df_group)
            print('target:',target)   
            print('add_idx_final:',add_idx_final)
            print()
        n -= 1

    print('--------------final swap idx------------------')
    print('add_idx_final:', add_idx_final)
    print('remove_idx:',remove_idx)

    print('-----------------after swap--------------------')
    # swap for high SKUs
    for (i,j) in zip(add_idx_final, remove_idx):
        #temp = df_temp_H[[i],[3,4,5]]
        df.iloc[i,7] = df.iloc[j,4]
        df.iloc[i,8] = df.iloc[j,5]
        df.iloc[i,9] = df.iloc[j,6]
        df.iloc[j,7] = df.iloc[i,4]
        df.iloc[j,8] = df.iloc[i,5]
        df.iloc[j,9] = df.iloc[i,6]
        df.iloc[i,10] = 'H'
        df.iloc[j,10] = 'H'
    #df = pd.Datarame(df)
    #df.columns = col
    df_group = df.groupby(by=('NewWorker-H'))['Qty'].sum()
    print(df_group)
    #print((abs(avg_1-df_group['1A'])+abs(avg_1-df_group['1B'])+abs(avg_1-df_group['1C'])+abs(avg_1-df_group['1D'])+\
    #     abs(avg_1-df_group['1E'])+abs(avg_1-df_group['1F'])+abs(avg_1-df_group['1G'])+abs(avg_1-df_group['1H']))/(line1_workers))

    df.to_csv('Bin_H.csv',index=False)

In [7]:
# medium high SKUs
def move_MH():
    df = pd.read_csv('Bin_H.csv')
    # insert columns: NewWorker, NewBin, NewBinRank, Change
    df.insert(11,'NewWorker-MH',0)
    df.insert(12,'NewBin-MH',0)
    df.insert(13,'NewBinRank-MH',0)
    df.insert(14,'Change-MH',0)

    for row in range(df.shape[0]):
        df.iloc[row,11] = df.iloc[row,7]
        df.iloc[row,12] = df.iloc[row,8]
        df.iloc[row,13] = df.iloc[row,9]
        df.iloc[row,14] = 0
    
    add_idx = df[(df['NewBinRank-H']!='medium high')&(df['Qty']<H_value)&(df['Qty']>=MH_value)].index.tolist()
    remove_idx = df[(df['NewBinRank-H']=='medium high')&(df['Qty']<MH_value)].index.tolist()
    print('add_idx:   ',add_idx)
    print('Total add:',len(add_idx),'\n')
    print('remove_idx:',remove_idx)
    print('Total remove:',len(remove_idx),'\n')

    pop_n = abs(len(add_idx) - len(remove_idx))
    while pop_n != 0:
        if len(add_idx) > len(remove_idx):
            add_idx.pop()
            pop_n -= 1
        if len(add_idx) < len(remove_idx):
            remove_idx.pop()
            pop_n -= 1
    print('add_idx:   ',add_idx)
    print('Total add:',len(add_idx),'\n')
    print('remove_idx:',remove_idx)
    print('Total remove:',len(remove_idx),'\n') 

    # find most suitable bin for medium high SKUs
    add_idx_final = []

    avg_1 = df[df['Qty']>=MH_value]['Qty'].sum() / (line1_workers + ratio * line2_workers)  #avg for (H_value+MH_value)/(8+ratio*5)
    avg_2 = ratio * avg_1
    print('line1 avg:',avg_1,'\n')
    print('line2 avg:',avg_2,'\n')

    df_group = df[df['Qty']>=MH_value].groupby(by=('NewWorker-H'))['Qty'].sum()
    print(df_group)


    target = (abs(avg_1-df_group['1A'])+abs(avg_1-df_group['1B'])+abs(avg_1-df_group['1C'])+abs(avg_1-df_group['1D'])+\
             abs(avg_1-df_group['1E'])+abs(avg_1-df_group['1F'])+abs(avg_1-df_group['1G'])+abs(avg_1-df_group['1H']))/line1_workers\
             +(abs(avg_2-df_group['2AA'])+abs(avg_2-df_group['2BB'])+abs(avg_2-df_group['2CC'])+abs(avg_2-df_group['2DD'])+\
             abs(avg_2-df_group['2EE']))/line2_workers
    print('target:',target,'\n')

    n = combination_n
    while n > 0:
        np.random.shuffle(add_idx)
        #print('remove_idx',remove_idx)
    
        for (i,j) in zip(add_idx,remove_idx):
            df.iloc[i,11] = df.iloc[j,7]
            df.iloc[i,12] = df.iloc[j,8]
            df.iloc[i,13] = df.iloc[j,9]
            df.iloc[j,11] = df.iloc[i,7]
            df.iloc[j,12] = df.iloc[i,8]
            df.iloc[j,13] = df.iloc[i,9]
    
        df_group = df[df['Qty']>=MH_value].groupby(by=('NewWorker-MH'))['Qty'].sum()
        #print(df_group)
        target_temp = (abs(avg_1-df_group['1A'])+abs(avg_1-df_group['1B'])+abs(avg_1-df_group['1C'])+abs(avg_1-df_group['1D'])+\
                      abs(avg_1-df_group['1E'])+abs(avg_1-df_group['1F'])+abs(avg_1-df_group['1G'])+abs(avg_1-df_group['1H']))/line1_workers\
                      +(abs(avg_2-df_group['2AA'])+abs(avg_2-df_group['2BB'])+abs(avg_2-df_group['2CC'])+abs(avg_2-df_group['2DD'])+\
                      abs(avg_2-df_group['2EE']))/line2_workers
        #print('target_temp:',target_temp)
        if target_temp < target:
            target = target_temp
            #print('target:',target)
            add_idx_final = add_idx.copy()
            #print('add_idx_final:',add_idx_final)
            print(df_group)
            print('target:',target)   
            print('add_idx_final:',add_idx_final)
            print()
        n -= 1

    print('------------final swap idx----------------')
    print('add_idx_final:', add_idx_final)
    print('remove_idx:',remove_idx)

    print('-----------------after swap--------------------')
    # swap for medium high SKUs
    for (i,j) in zip(add_idx_final, remove_idx):
        #temp = df_temp_H[[i],[3,4,5]]
        df.iloc[i,11] = df.iloc[j,7]
        df.iloc[i,12] = df.iloc[j,8]
        df.iloc[i,13] = df.iloc[j,9]
        df.iloc[j,11] = df.iloc[i,7]
        df.iloc[j,12] = df.iloc[i,8]
        df.iloc[j,13] = df.iloc[i,9]
        df.iloc[i,14] = 'MH'
        df.iloc[j,14] = 'MH'
    #df = pd.Datarame(df)
    #df.columns = col
    df_group = df.groupby(by=('NewWorker-MH'))['Qty'].sum()
    print(df_group)
    #print((abs(avg_1-df_group['1A'])+abs(avg_1-df_group['1B'])+abs(avg_1-df_group['1C'])+abs(avg_1-df_group['1D'])+\
    #       abs(avg_1-df_group['1E'])+abs(avg_1-df_group['1F'])+abs(avg_1-df_group['1G'])+abs(avg_1-df_group['1H']))/line1_workers\
    #       +(abs(avg_2-df_group['2AA'])+abs(avg_2-df_group['2BB'])+abs(avg_2-df_group['2CC'])+abs(avg_2-df_group['2DD'])+\
    #       abs(avg_2-df_group['2EE']))/line2_workers)

    df.to_csv('Bin_MH.csv',index=False)

In [8]:
# medium SKUs
def move_M():
    df = pd.read_csv('Bin_MH.csv')
    # insert columns: NewWorker, NewBin, NewBinRank, Change
    df.insert(15,'NewWorker-M',0)
    df.insert(16,'NewBin-M',0)
    df.insert(17,'NewBinRank-M',0)
    df.insert(18,'Change-M',0)

    for row in range(df.shape[0]):
        df.iloc[row,15] = df.iloc[row,11]
        df.iloc[row,16] = df.iloc[row,12]
        df.iloc[row,17] = df.iloc[row,13]
        df.iloc[row,18] = 0
    
    add_idx = df[(df['NewBinRank-MH']!='medium')&(df['Qty']<MH_value)&(df['Qty']>=M_value)].index.tolist()
    remove_idx = df[(df['NewBinRank-MH']=='medium')&((df['Qty']<M_value) | (df['Qty']>=MH_value))].index.tolist()
    print('add_idx:   ',add_idx)
    print('Total add:',len(add_idx),'\n')
    print('remove_idx:',remove_idx)
    print('Total remove:',len(remove_idx),'\n')

    pop_n = abs(len(add_idx) - len(remove_idx))
    while pop_n != 0:
        if len(add_idx) > len(remove_idx):
            add_idx.pop()
            pop_n -= 1
        if len(add_idx) < len(remove_idx):
            remove_idx.pop()
            pop_n -= 1
    print('add_idx:   ',add_idx)
    print('Total add:',len(add_idx),'\n')
    print('remove_idx:',remove_idx)
    print('Total remove:',len(remove_idx),'\n') 

    # find most suitable bin for medium SKUs
    add_idx_final = []

    avg_1 = df[df['Qty']>=M_value]['Qty'].sum() / (line1_workers + ratio * line2_workers)  
    avg_2 = ratio * avg_1
    print('line1 avg:',avg_1,'\n')
    print('line2 avg:',avg_2,'\n')

    df_group = df[df['Qty']>=M_value].groupby(by=('NewWorker-MH'))['Qty'].sum()
    print(df_group)


    target = (abs(avg_1-df_group['1A'])+abs(avg_1-df_group['1B'])+abs(avg_1-df_group['1C'])+abs(avg_1-df_group['1D'])+\
             abs(avg_1-df_group['1E'])+abs(avg_1-df_group['1F'])+abs(avg_1-df_group['1G'])+abs(avg_1-df_group['1H']))/line1_workers\
             +(abs(avg_2-df_group['2AA'])+abs(avg_2-df_group['2BB'])+abs(avg_2-df_group['2CC'])+abs(avg_2-df_group['2DD'])+\
             abs(avg_2-df_group['2EE']))/line2_workers
    print('target:',target,'\n')

    n = combination_n
    while n > 0:
        np.random.shuffle(add_idx)
        #print('remove_idx',remove_idx)
    
        for (i,j) in zip(add_idx,remove_idx):
            df.iloc[i,15] = df.iloc[j,11]
            df.iloc[i,16] = df.iloc[j,12]
            df.iloc[i,17] = df.iloc[j,13]
            df.iloc[j,15] = df.iloc[i,11]
            df.iloc[j,16] = df.iloc[i,12]
            df.iloc[j,17] = df.iloc[i,13]
    
        df_group = df[df['Qty']>=M_value].groupby(by=('NewWorker-M'))['Qty'].sum()
        #print(df_group)
        target_temp = (abs(avg_1-df_group['1A'])+abs(avg_1-df_group['1B'])+abs(avg_1-df_group['1C'])+abs(avg_1-df_group['1D'])+\
                      abs(avg_1-df_group['1E'])+abs(avg_1-df_group['1F'])+abs(avg_1-df_group['1G'])+abs(avg_1-df_group['1H']))/line1_workers\
                      +(abs(avg_2-df_group['2AA'])+abs(avg_2-df_group['2BB'])+abs(avg_2-df_group['2CC'])+abs(avg_2-df_group['2DD'])+\
                      abs(avg_2-df_group['2EE']))/line2_workers
        #print('target_temp:',target_temp)
        if target_temp < target:
            target = target_temp
            #print('target:',target)
            add_idx_final = add_idx.copy()
            #print('add_idx_final:',add_idx_final)
            print(df_group)
            print('target:',target)   
            print('add_idx_final:',add_idx_final)
            print()
        n -= 1
    
    print('------------final swap idx----------------')
    print('add_idx_final:', add_idx_final)
    print('remove_idx:',remove_idx)

    print('-----------------after swap--------------------')
    # swap for medium SKUs
    for (i,j) in zip(add_idx_final, remove_idx):
        #temp = df_temp_H[[i],[3,4,5]]
        df.iloc[i,15] = df.iloc[j,11]
        df.iloc[i,16] = df.iloc[j,12]
        df.iloc[i,17] = df.iloc[j,13]
        df.iloc[j,15] = df.iloc[i,11]
        df.iloc[j,16] = df.iloc[i,12]
        df.iloc[j,17] = df.iloc[i,13]
        df.iloc[i,18] = 'M'
        df.iloc[j,18] = 'M'
    #df = pd.Datarame(df)
    #df.columns = col
    df_group = df.groupby(by=('NewWorker-MH'))['Qty'].sum()
    print(df_group)
    #print((abs(avg_1-df_group['1A'])+abs(avg_1-df_group['1B'])+abs(avg_1-df_group['1C'])+abs(avg_1-df_group['1D'])+\
    #       abs(avg_1-df_group['1E'])+abs(avg_1-df_group['1F'])+abs(avg_1-df_group['1G'])+abs(avg_1-df_group['1H']))/line1_workers\
    #       +(abs(avg_2-df_group['2AA'])+abs(avg_2-df_group['2BB'])+abs(avg_2-df_group['2CC'])+abs(avg_2-df_group['2DD'])+\
    #       abs(avg_2-df_group['2EE']))/line2_workers)

    df.to_csv('Bin_M.csv',index=False)

In [9]:
# medium low SKUs
def move_ML():
    df = pd.read_csv('Bin_M.csv')
    # insert columns: NewWorker, NewBin, NewBinRank, Change
    df.insert(19,'NewWorker-ML',0)
    df.insert(20,'NewBin-ML',0)
    df.insert(21,'NewBinRank-ML',0)
    df.insert(22,'Change-ML',0)

    for row in range(df.shape[0]):
        df.iloc[row,19] = df.iloc[row,15]
        df.iloc[row,20] = df.iloc[row,16]
        df.iloc[row,21] = df.iloc[row,17]
        df.iloc[row,22] = 0
    
    add_idx = df[(df['NewBinRank-M']!='medium low')&(df['Qty']<M_value)&(df['Qty']>=ML_value)].index.tolist()
    remove_idx = df[(df['NewBinRank-M']=='medium low')&((df['Qty']<ML_value) | (df['Qty']>=M_value))].index.tolist()
    print('add_idx:   ',add_idx)
    print('Total add:',len(add_idx),'\n')
    print('remove_idx:',remove_idx)
    print('Total remove:',len(remove_idx),'\n')

    pop_n = abs(len(add_idx) - len(remove_idx))
    while pop_n != 0:
        if len(add_idx) > len(remove_idx):
            add_idx.pop()
            pop_n -= 1
        if len(add_idx) < len(remove_idx):
            remove_idx.pop()
            pop_n -= 1
    print('add_idx:   ',add_idx)
    print('Total add:',len(add_idx),'\n')
    print('remove_idx:',remove_idx)
    print('Total remove:',len(remove_idx),'\n') 

    # find most suitable bin for medium SKUs
    add_idx_final = []

    avg_1 = df[df['Qty']>=ML_value]['Qty'].sum() / (line1_workers + ratio * line2_workers)  
    avg_2 = ratio * avg_1
    print('line1 avg:',avg_1,'\n')
    print('line2 avg:',avg_2,'\n')

    df_group = df[df['Qty']>=ML_value].groupby(by=('NewWorker-M'))['Qty'].sum()
    print(df_group)


    target = (abs(avg_1-df_group['1A'])+abs(avg_1-df_group['1B'])+abs(avg_1-df_group['1C'])+abs(avg_1-df_group['1D'])+\
             abs(avg_1-df_group['1E'])+abs(avg_1-df_group['1F'])+abs(avg_1-df_group['1G'])+abs(avg_1-df_group['1H']))/line1_workers\
             +(abs(avg_2-df_group['2AA'])+abs(avg_2-df_group['2BB'])+abs(avg_2-df_group['2CC'])+abs(avg_2-df_group['2DD'])+\
             abs(avg_2-df_group['2EE']))/line2_workers
    print('target:',target,'\n')

    n = combination_n
    while n > 0:
        np.random.shuffle(add_idx)
        #print('remove_idx',remove_idx)
    
        for (i,j) in zip(add_idx,remove_idx):
            df.iloc[i,19] = df.iloc[j,15]
            df.iloc[i,20] = df.iloc[j,16]
            df.iloc[i,21] = df.iloc[j,17]
            df.iloc[j,19] = df.iloc[i,15]
            df.iloc[j,20] = df.iloc[i,16]
            df.iloc[j,21] = df.iloc[i,17]
    
        df_group = df[df['Qty']>=M_value].groupby(by=('NewWorker-M'))['Qty'].sum()
        #print(df_group)
        target_temp = (abs(avg_1-df_group['1A'])+abs(avg_1-df_group['1B'])+abs(avg_1-df_group['1C'])+abs(avg_1-df_group['1D'])+\
                      abs(avg_1-df_group['1E'])+abs(avg_1-df_group['1F'])+abs(avg_1-df_group['1G'])+abs(avg_1-df_group['1H']))/line1_workers\
                      +(abs(avg_2-df_group['2AA'])+abs(avg_2-df_group['2BB'])+abs(avg_2-df_group['2CC'])+abs(avg_2-df_group['2DD'])+\
                      abs(avg_2-df_group['2EE']))/line2_workers
        #print('target_temp:',target_temp)
        if target_temp < target:
            target = target_temp
            #print('target:',target)
            add_idx_final = add_idx.copy()
            #print('add_idx_final:',add_idx_final)
            print(df_group)
            print('target:',target)   
            print('add_idx_final:',add_idx_final)
            print()
        n -= 1
    
    print('------------final swap idx----------------')
    print('add_idx_final:', add_idx_final)
    print('remove_idx:',remove_idx)

    print('-----------------after swap--------------------')
    # swap for medium low SKUs
    for (i,j) in zip(add_idx_final, remove_idx):
        #temp = df_temp_H[[i],[3,4,5]]
        df.iloc[i,19] = df.iloc[j,15]
        df.iloc[i,20] = df.iloc[j,16]
        df.iloc[i,21] = df.iloc[j,17]
        df.iloc[j,19] = df.iloc[i,15]
        df.iloc[j,20] = df.iloc[i,16]
        df.iloc[j,21] = df.iloc[i,17]
        df.iloc[i,22] = 'ML'
        df.iloc[j,22] = 'ML'
    #df = pd.Datarame(df)
    #df.columns = col
    df_group = df.groupby(by=('NewWorker-M'))['Qty'].sum()
    print(df_group)
    #print((abs(avg_1-df_group['1A'])+abs(avg_1-df_group['1B'])+abs(avg_1-df_group['1C'])+abs(avg_1-df_group['1D'])+\
    #       abs(avg_1-df_group['1E'])+abs(avg_1-df_group['1F'])+abs(avg_1-df_group['1G'])+abs(avg_1-df_group['1H']))/line1_workers\
    #       +(abs(avg_2-df_group['2AA'])+abs(avg_2-df_group['2BB'])+abs(avg_2-df_group['2CC'])+abs(avg_2-df_group['2DD'])+\
    #       abs(avg_2-df_group['2EE']))/line2_workers)

    df.to_csv('Bin_ML.csv',index=False)

In [ ]:
l1 = tk.Label(window, text='Open Sales Order .xlsx or .csv')
l1.pack()
b1 = tk.Button(window,text='Browse',command=open_file1)
b1.pack()

l2  = tk.Label(window, text='Open Bin Mat .xlsx')
l2.pack()
b2 = tk.Button(window,text='Browse',command=open_file2)
b2.pack()

l3  = tk.Label(window, text='Open Expedite Report .xlsx')
l3.pack()
b3 = tk.Button(window,text='Browse',command=open_file3)
b3.pack()

b4 = tk.Button(window, text='Move High Only', command=submit_move_H)
b4.pack()

b5 = tk.Button(window, text='Move High and Medium High', command=submit_move_MH)
b5.pack()

b6 = tk.Button(window, text='Move High/MH and Medium', command=submit_move_M)
b6.pack()

b7 = tk.Button(window, text='Move High/MH and M/ML', command=submit_move_ML)
b7.pack()

window.mainloop()    

['Worker', 'Bin', 'BinRank', 'Material', 'Qty', 'Description', 'Volume']
['Material', 'Description', 'Volume', 'Qty', 'Worker', 'Bin', 'BinRank']
High:         32 bins
Medium_High:  72 bins
Medium:      130 bins
Medium_Low:    0 bins
Low:         534 bins

High:        Qty>= 219.0
Medium_High: Qty>= 106.0
Medium:      Qty>= 37.0
Medium_Low:  Qty>= 37.0
Low:         Qty< 37.0
add_idx:    [51, 66, 75, 87, 150, 166, 216, 253, 314, 315, 430, 464, 468, 507, 511, 514, 527, 528, 536, 583, 591, 599, 650, 655]
Total add: 24 

remove_idx: [28, 30, 31, 77, 78, 79, 124, 126, 127, 172, 173, 174, 220, 221, 222, 268, 270, 316, 318, 319, 364, 365, 366, 367]
Total remove: 24 

add_idx:    [51, 66, 75, 87, 150, 166, 216, 253, 314, 315, 430, 464, 468, 507, 511, 514, 527, 528, 536, 583, 591, 599, 650, 655]
Total add: 24 

remove_idx: [28, 30, 31, 77, 78, 79, 124, 126, 127, 172, 173, 174, 220, 221, 222, 268, 270, 316, 318, 319, 364, 365, 366, 367]
Total remove: 24 

Worker
1A      282.0
1B     2506.0
1C   

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\chenmin\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\indexes\base.py", line 4381, in get_value
    return libindex.get_value_box(s, key)
  File "pandas\_libs\index.pyx", line 52, in pandas._libs.index.get_value_box
  File "pandas\_libs\index.pyx", line 48, in pandas._libs.index.get_value_at
  File "pandas\_libs\util.pxd", line 113, in pandas._libs.util.get_value_at
  File "pandas\_libs\util.pxd", line 98, in pandas._libs.util.validate_indexer
TypeError: 'str' object cannot be interpreted as an integer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\chenmin\appdata\local\programs\python\python37-32\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-4-bd9f0e09aa39>", line 74, in submit_move_ML
    move_H()
  File "<ipython-input-6-5933a39eb5e8>", line 33, in m